In [2]:
import numpy as np
np.set_printoptions(precision = 3, linewidth = 150)

In [3]:
np.random.seed(4)
counts = np.repeat(
    np.arange(5, 10),
    [8, 4, 4, 3, 1])
data = np.random.poisson(counts[None, :], [100, 20]).astype(np.float32)
data -= np.mean(data, axis=0)
cov = np.cov(data.T).astype(np.float32)
print('{} {}'.format(cov.shape, cov.dtype))

(20, 20) float32


In [4]:
A = np.diag(np.linalg.eigh(cov[:8, :8])[0])
A_n = np.c_[np.r_[np.sqrt(A), np.zeros_like(A[0:1, :])], np.zeros([A.shape[0]+1,1])]
A_n

array([[1.646, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 1.762, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 1.966, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 2.06 , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 2.176, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 2.313, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 2.411, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 2.569, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ]])

In [5]:
from l0pca.bunch import rank_one_update

2022-11-13 07:44:14.337738: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-13 07:44:14.424924: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-13 07:44:14.447651: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-13 07:44:14.788378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [6]:
update_aug = rank_one_update.create_update_append_column(
    np.arange(8), np.sqrt(np.diagonal(A)),
    np.linalg.eigh(cov[:8, :8])[1], 8, cov,
    np.linalg.norm(data, axis=0), 100).numpy()
update_aug = np.concatenate([update_aug[1:], update_aug[0:1]])
update_aug

2022-11-13 07:44:15.300295: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-13 07:44:15.315160: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-13 07:44:15.315867: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-13 07:44:15.316697: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow 

array([-0.277,  0.369,  0.099, -0.182,  0.116, -0.006,  0.37 , -0.273,  2.23 ], dtype=float32)

In [7]:
A_aug_full = A_n.copy()
A_aug_full[:, -1] = update_aug
A_aug_full

array([[ 1.646,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.277],
       [ 0.   ,  1.762,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.369],
       [ 0.   ,  0.   ,  1.966,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.099],
       [ 0.   ,  0.   ,  0.   ,  2.06 ,  0.   ,  0.   ,  0.   ,  0.   , -0.182],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  2.176,  0.   ,  0.   ,  0.   ,  0.116],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  2.313,  0.   ,  0.   , -0.006],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  2.411,  0.   ,  0.37 ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  2.569, -0.273],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  2.23 ]])

In [8]:
np.allclose(
    A_n ** 2 + update_aug[:, None] * update_aug[None, :],
    A_aug_full.dot(A_aug_full.T))

True

In [9]:
import scipy.linalg
V = scipy.linalg.block_diag(
    np.linalg.eigh(cov[:8, :8])[1],
    [[1.]]
)
np.allclose(np.linalg.inv(V), V.T)

True

In [10]:
V.dot(A_n ** 2).dot(V.T)

array([[ 4.614, -0.122,  0.111,  0.114,  0.107, -0.435,  0.381,  0.306,  0.   ],
       [-0.122,  3.981, -0.14 ,  0.112,  0.508, -0.22 ,  0.381, -0.223,  0.   ],
       [ 0.111, -0.14 ,  4.991, -0.181, -0.108, -0.018, -0.447,  0.232,  0.   ],
       [ 0.114,  0.112, -0.181,  3.826, -0.44 ,  0.688,  0.74 , -0.498,  0.   ],
       [ 0.107,  0.508, -0.108, -0.44 ,  4.499, -1.116, -0.232, -0.686,  0.   ],
       [-0.435, -0.22 , -0.018,  0.688, -1.116,  5.277, -0.843, -0.234,  0.   ],
       [ 0.381,  0.381, -0.447,  0.74 , -0.232, -0.843,  4.29 , -0.207,  0.   ],
       [ 0.306, -0.223,  0.232, -0.498, -0.686, -0.234, -0.207,  4.937,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [11]:
cov[:9, :9]

array([[ 4.614, -0.122,  0.111,  0.114,  0.107, -0.435,  0.381,  0.306,  0.065],
       [-0.122,  3.981, -0.14 ,  0.112,  0.508, -0.22 ,  0.381, -0.223,  0.348],
       [ 0.111, -0.14 ,  4.991, -0.181, -0.108, -0.018, -0.447,  0.232,  0.317],
       [ 0.114,  0.112, -0.181,  3.826, -0.44 ,  0.688,  0.74 , -0.498, -1.116],
       [ 0.107,  0.508, -0.108, -0.44 ,  4.499, -1.116, -0.232, -0.686,  0.017],
       [-0.435, -0.22 , -0.018,  0.688, -1.116,  5.277, -0.843, -0.234, -0.588],
       [ 0.381,  0.381, -0.447,  0.74 , -0.232, -0.843,  4.29 , -0.207,  0.275],
       [ 0.306, -0.223,  0.232, -0.498, -0.686, -0.234, -0.207,  4.937,  0.517],
       [ 0.065,  0.348,  0.317, -1.116,  0.017, -0.588,  0.275,  0.517,  5.452]], dtype=float32)

In [12]:
V.dot(A_n ** 2 + update_aug[:, None].dot(update_aug[None, :])).dot(V.T)

array([[ 4.614, -0.119,  0.113,  0.104,  0.107, -0.439,  0.384,  0.31 ,  0.04 ],
       [-0.119,  4.013, -0.116,  0.01 ,  0.502, -0.255,  0.412, -0.187,  0.4  ],
       [ 0.113, -0.116,  5.01 , -0.259, -0.113, -0.045, -0.424,  0.259,  0.306],
       [ 0.104,  0.01 , -0.259,  4.147, -0.423,  0.799,  0.644, -0.611, -1.263],
       [ 0.107,  0.502, -0.113, -0.423,  4.5  , -1.109, -0.238, -0.692, -0.069],
       [-0.439, -0.255, -0.045,  0.799, -1.109,  5.316, -0.877, -0.273, -0.438],
       [ 0.384,  0.412, -0.424,  0.644, -0.238, -0.877,  4.319, -0.173,  0.377],
       [ 0.31 , -0.187,  0.259, -0.611, -0.692, -0.273, -0.173,  4.977,  0.445],
       [ 0.04 ,  0.4  ,  0.306, -1.263, -0.069, -0.438,  0.377,  0.445,  4.971]])

In [13]:
V.dot(A_aug_full.dot(A_aug_full.T)).dot(V.T)

array([[ 4.614, -0.119,  0.113,  0.104,  0.107, -0.439,  0.384,  0.31 ,  0.04 ],
       [-0.119,  4.013, -0.116,  0.01 ,  0.502, -0.255,  0.412, -0.187,  0.4  ],
       [ 0.113, -0.116,  5.01 , -0.259, -0.113, -0.045, -0.424,  0.259,  0.306],
       [ 0.104,  0.01 , -0.259,  4.147, -0.423,  0.799,  0.644, -0.611, -1.263],
       [ 0.107,  0.502, -0.113, -0.423,  4.5  , -1.109, -0.238, -0.692, -0.069],
       [-0.439, -0.255, -0.045,  0.799, -1.109,  5.316, -0.877, -0.273, -0.438],
       [ 0.384,  0.412, -0.424,  0.644, -0.238, -0.877,  4.319, -0.173,  0.377],
       [ 0.31 , -0.187,  0.259, -0.611, -0.692, -0.273, -0.173,  4.977,  0.445],
       [ 0.04 ,  0.4  ,  0.306, -1.263, -0.069, -0.438,  0.377,  0.445,  4.971]])

In [14]:
np.linalg.eigh(A_aug_full.dot(A_aug_full.T))[1].T.dot(V.T).T

array([[ 1.835e-04,  1.401e-01, -9.153e-02,  7.244e-01,  7.161e-02,  4.679e-01, -4.414e-01, -6.150e-02,  1.570e-01],
       [ 2.488e-02,  3.872e-01, -8.035e-01, -4.180e-02,  2.572e-01, -2.639e-01,  6.425e-02, -1.982e-01,  1.509e-01],
       [-8.242e-02, -4.495e-02, -4.065e-02, -2.471e-01,  6.353e-01,  5.449e-01,  2.557e-01,  3.869e-01,  1.133e-01],
       [ 6.454e-01, -3.669e-01, -1.479e-01, -1.569e-01,  2.374e-01,  1.217e-01, -2.872e-01, -2.695e-01, -4.196e-01],
       [-1.346e-01, -6.239e-01, -2.381e-01,  1.662e-01, -1.589e-01,  1.893e-01,  4.537e-01, -4.065e-01,  2.760e-01],
       [-3.588e-01, -2.789e-01, -2.278e-01,  3.844e-01,  1.551e-01, -2.460e-01,  8.480e-02,  3.318e-01, -6.262e-01],
       [-5.439e-01, -2.480e-01,  1.166e-01, -2.399e-01,  3.628e-01, -1.803e-01, -5.306e-01, -3.210e-01,  1.512e-01],
       [-1.139e-02, -3.030e-01, -4.193e-01, -2.577e-01, -4.251e-01,  9.297e-02, -3.983e-01,  5.208e-01,  2.160e-01],
       [ 3.651e-01, -2.709e-01,  1.562e-01,  2.976e-01,  3.272e-

In [15]:
guess = np.random.normal(size=[9, 1])
guess /= np.linalg.norm(guess)
guess

array([[ 0.048],
       [-0.045],
       [ 0.188],
       [-0.251],
       [-0.249],
       [-0.487],
       [-0.73 ],
       [-0.072],
       [-0.245]])

In [16]:
np.linalg.eigh(A_aug_full.dot(A_aug_full.T))

(array([2.578, 2.951, 3.837, 4.13 , 4.638, 4.983, 5.348, 6.242, 7.158]),
 array([[ 8.790e-01,  4.117e-01, -6.601e-02, -1.074e-01, -9.766e-02,  1.393e-01, -2.259e-03, -6.394e-02, -9.490e-02],
        [-2.954e-01,  8.658e-01,  1.352e-01,  1.980e-01,  1.635e-01, -2.245e-01,  3.538e-03,  9.588e-02,  1.387e-01],
        [-3.250e-02,  3.893e-02, -9.664e-01,  2.056e-01,  8.716e-02, -1.014e-01,  1.439e-03,  3.402e-02,  4.591e-02],
        [ 4.611e-02, -5.052e-02,  1.204e-01,  8.881e-01, -3.135e-01,  2.814e-01, -3.547e-03, -7.431e-02, -9.526e-02],
        [-2.271e-02,  2.334e-02, -3.479e-02, -1.061e-01, -8.342e-01, -5.303e-01,  4.059e-03,  6.272e-02,  7.296e-02],
        [ 9.059e-04, -8.899e-04,  1.058e-03,  2.692e-03,  5.701e-03, -1.864e-02, -9.998e-01, -5.431e-03, -5.014e-03],
        [-4.822e-02,  4.633e-02, -5.028e-02, -1.211e-01, -2.141e-01,  5.093e-01, -1.711e-02,  7.047e-01,  4.197e-01],
        [ 2.869e-02, -2.688e-02,  2.661e-02,  6.106e-02,  9.492e-02, -1.937e-01,  4.713e-03,  6.280e-

In [25]:
lam = np.linalg.eigh(cov[:9, :9])[0][7]
print((lam, np.linalg.eigh(cov[:9, :9])[0]))

(6.242337, array([2.578, 2.951, 3.837, 4.13 , 4.638, 4.983, 5.348, 6.242, 7.158], dtype=float32))


In [18]:
g = np.linalg.solve(A_aug_full, guess)
g

array([[ 0.011],
       [-0.002],
       [ 0.101],
       [-0.132],
       [-0.108],
       [-0.211],
       [-0.286],
       [-0.04 ],
       [-0.11 ]])

In [19]:
mat = A_aug_full.T - np.linalg.inv(A_aug_full) * lam
mat

array([[-2.145,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.47 ],
       [ 0.   , -1.781,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.586],
       [ 0.   ,  0.   , -1.21 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.141],
       [ 0.   ,  0.   ,  0.   , -0.97 ,  0.   ,  0.   ,  0.   ,  0.   , -0.247],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.694,  0.   ,  0.   ,  0.   ,  0.149],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.387,  0.   ,  0.   , -0.007],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.178,  0.   ,  0.429],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.138, -0.298],
       [-0.277,  0.369,  0.099, -0.182,  0.116, -0.006,  0.37 , -0.273, -0.57 ]])

In [20]:
e2 = np.linalg.solve(mat, g)
e2 /= np.linalg.norm(e2)
e2

array([[ 0.064],
       [-0.096],
       [-0.034],
       [ 0.074],
       [-0.063],
       [ 0.005],
       [-0.705],
       [-0.628],
       [-0.292]])

In [21]:
e3 = np.linalg.solve(mat, np.linalg.solve(A_aug_full, e2))
e3 /= np.linalg.norm(e3)
e3

array([[ 0.064],
       [-0.096],
       [-0.034],
       [ 0.074],
       [-0.063],
       [ 0.005],
       [-0.705],
       [-0.628],
       [-0.292]])

In [22]:
np.linalg.eigh(cov[:9, :9])[1][:, 7]

array([ 0.062,  0.211, -0.378,  0.228,  0.403, -0.343,  0.334, -0.508, -0.327], dtype=float32)

In [23]:
e2x = np.linalg.solve(
    A_aug_full.dot(A_aug_full.T)
    - np.diag(np.repeat(lam, 9)),
    guess)
e2x /= np.linalg.norm(e2x)
e2x

array([[ 0.064],
       [-0.096],
       [-0.034],
       [ 0.074],
       [-0.063],
       [ 0.005],
       [-0.705],
       [-0.628],
       [-0.292]])

In [24]:
np.linalg.eigh(A_aug_full.dot(A_aug_full.T)
    - np.diag(np.repeat(lam, 9)))

(array([-3.665e+00, -3.291e+00, -2.405e+00, -2.112e+00, -1.604e+00, -1.259e+00, -8.939e-01,  2.870e-07,  9.158e-01]),
 array([[ 8.790e-01,  4.117e-01,  6.601e-02,  1.074e-01, -9.766e-02,  1.393e-01, -2.259e-03,  6.394e-02, -9.490e-02],
        [-2.954e-01,  8.658e-01, -1.352e-01, -1.980e-01,  1.635e-01, -2.245e-01,  3.538e-03, -9.588e-02,  1.387e-01],
        [-3.250e-02,  3.893e-02,  9.664e-01, -2.056e-01,  8.716e-02, -1.014e-01,  1.439e-03, -3.402e-02,  4.591e-02],
        [ 4.611e-02, -5.052e-02, -1.204e-01, -8.881e-01, -3.135e-01,  2.814e-01, -3.547e-03,  7.431e-02, -9.526e-02],
        [-2.271e-02,  2.334e-02,  3.479e-02,  1.061e-01, -8.342e-01, -5.303e-01,  4.059e-03, -6.272e-02,  7.296e-02],
        [ 9.059e-04, -8.899e-04, -1.058e-03, -2.692e-03,  5.701e-03, -1.864e-02, -9.998e-01,  5.431e-03, -5.014e-03],
        [-4.822e-02,  4.633e-02,  5.028e-02,  1.211e-01, -2.141e-01,  5.093e-01, -1.711e-02, -7.047e-01,  4.197e-01],
        [ 2.869e-02, -2.688e-02, -2.661e-02, -6.106e-02,

In [33]:
np.linalg.eigh(A_aug_full.dot(A_aug_full.T))[1].T.dot(
    np.r_[
        np.c_[np.linalg.eigh(cov[:8, :8])[1].T, np.zeros((8, 1))],
        np.c_[np.zeros((1, 8)), [[1.]]]
    ]).T

array([[ 1.835e-04,  1.401e-01, -9.153e-02,  7.244e-01,  7.161e-02,  4.679e-01, -4.414e-01, -6.150e-02,  1.570e-01],
       [ 2.488e-02,  3.872e-01, -8.035e-01, -4.180e-02,  2.572e-01, -2.639e-01,  6.425e-02, -1.982e-01,  1.509e-01],
       [-8.242e-02, -4.495e-02, -4.065e-02, -2.471e-01,  6.353e-01,  5.449e-01,  2.557e-01,  3.869e-01,  1.133e-01],
       [ 6.454e-01, -3.669e-01, -1.479e-01, -1.569e-01,  2.374e-01,  1.217e-01, -2.872e-01, -2.695e-01, -4.196e-01],
       [-1.346e-01, -6.239e-01, -2.381e-01,  1.662e-01, -1.589e-01,  1.893e-01,  4.537e-01, -4.065e-01,  2.760e-01],
       [-3.588e-01, -2.789e-01, -2.278e-01,  3.844e-01,  1.551e-01, -2.460e-01,  8.480e-02,  3.318e-01, -6.262e-01],
       [-5.439e-01, -2.480e-01,  1.166e-01, -2.399e-01,  3.628e-01, -1.803e-01, -5.306e-01, -3.210e-01,  1.512e-01],
       [-1.139e-02, -3.030e-01, -4.193e-01, -2.577e-01, -4.251e-01,  9.297e-02, -3.983e-01,  5.208e-01,  2.160e-01],
       [ 3.651e-01, -2.709e-01,  1.562e-01,  2.976e-01,  3.272e-

In [28]:
np.linalg.eigh(cov[:9, :9])[1]

array([[ 6.627e-04, -1.408e-01, -9.194e-02, -7.236e-01,  7.068e-02, -4.694e-01,  4.414e-01,  6.241e-02, -1.555e-01],
       [-1.154e-02, -3.975e-01, -8.099e-01,  5.409e-02,  2.433e-01,  2.418e-01, -6.386e-02,  2.114e-01, -1.287e-01],
       [ 9.167e-02,  3.780e-02, -4.508e-02,  2.557e-01,  6.256e-01, -5.605e-01, -2.554e-01, -3.775e-01, -9.757e-02],
       [-6.875e-01,  3.992e-01, -1.280e-01,  1.182e-01,  2.812e-01, -5.213e-02,  2.859e-01,  2.279e-01,  3.496e-01],
       [ 1.309e-01,  6.267e-01, -2.365e-01, -1.694e-01, -1.552e-01, -1.835e-01, -4.538e-01,  4.030e-01, -2.817e-01],
       [ 3.481e-01,  2.872e-01, -2.225e-01, -3.945e-01,  1.667e-01,  2.645e-01, -8.514e-02, -3.431e-01,  6.070e-01],
       [ 5.576e-01,  2.374e-01,  1.102e-01,  2.524e-01,  3.487e-01,  1.579e-01,  5.310e-01,  3.344e-01, -1.288e-01],
       [ 2.371e-02,  2.935e-01, -4.253e-01,  2.693e-01, -4.382e-01, -1.139e-01,  3.987e-01, -5.081e-01, -1.946e-01],
       [-2.629e-01,  2.088e-01,  1.372e-01, -2.713e-01,  3.160e-